## Imports

In [ ]:
import datetime as              dt
import pandas   as              pd
import csv
import numpy    as              np
from   glob    import           glob
import re
import os
from   io    import             StringIO
from pathlib import             Path, PurePosixPath, PureWindowsPath
import xml.etree.ElementTree as ET

## Module Data

#### Static Data.

In [ ]:
# The asof data for this version of the library.
asof = '2024-0406-1136'

### Test and Configuration Data
These cells setup any files used for testing the functions and classes in this module.  Configurations for testing on Linux/Colab, Linux/wsl2-Jupyter, Linux/Jupyter, and Windows/Jupyter

### All Systems

In [ ]:
# Set these for all systems.
if __name__ == "__main__":
  metashape_cal_filename = 'we-1500-A1-agisoft-camcal.xml'
  metashape_ref_filename = 'we-1500-A1-agisoft-camcal-ref.txt'
  inpho_filename         = 'we-1500-A1-inpho-camcal.txt'
  opencv_filename        = 'we-1500-A1-opencv-camcal.xml'
  ref_test_mask          = '*-ref.txt'

### Colab

In [ ]:
# Colab setup.
# See: https://openincolab.com/ to generate a Colab badge.
if __name__ == "__main__":
  if 'google.colab' not in str(get_ipython()):
    print('Not running on CoLab')
    IN_COLAB = False
  else:
    print('Running on Linux/CoLab')
    import google.colab
    IN_COLAB = True
    if os.path.exists('/content/drive') == False:
      from google.colab import drive
      drive.mount('/content/drive')
    else:
      print('Google Drive already mounted.')
      test_data_dir          = '/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/'
      metashape_cal_filename = test_data_dir + metashape_cal_filename
      metashape_ref_filename = test_data_dir + metashape_ref_filename
      inpho_filename         = test_data_dir + inpho_filename
      opencv_filename        = test_data_dir + opencv_filename
      print('\n**** Testing Files ****')
      print(f'{metashape_cal_filename    = }')
      print(f'{metashape_ref_filename    = }')
      print(f'{inpho_filename            = }')
      print(f'{opencv_filename           = }')
      print("\n**** Reference data test files ****")
      ref_test_files = glob(f'{test_data_dir}/{ref_test_mask}')
      for f in ref_test_files:
        print(f)

### Windows/Juypter

In [ ]:
# Windows setup.
if __name__ == "__main__" and os.name == 'nt':
  print('Running on Windows/Jupyter.')

### Linux/Jupyter

In [ ]:
# Linux/Jupyter setup.
if __name__ == "__main__" and os.name == 'posix' and not IN_COLAB:
  print('Running on Linux/Jupyter.')
  pass

## Functions

### class MetaShapeReference

In [ ]:
class MetaShapeReference:
  def __init__(self, filename):
    self.filename      = filename
    self.df            = None
    self.meta          = None
    self.total_error   = None
    self.data          = None

### def read_metashape_reference_file_into_dataframe()

In [ ]:
def read_metashape_reference_file_into_dataframe(
    filename           # Metashape reference filename to read.
    ) -> pd.DataFrame: # MetashapeReference class data structure.
  """
  Read a Metashape reference file into a MetashapeReference class
  and save the data in a Pandas dataframe.  Also save the filename, the
  metadata from the comment above the header line, the total_error as
  a Pandas Dataframe, and the contents of the specified file in string,
  in the returned data structure.
  """

  # Create the Matashape data structure, and save the filename in it.
  ref = MetaShapeReference( filename )

  # Extract the root filename
  ref.root_filename = PurePosixPath(filename).name

  # Read the file into a list of lines. so we can get rid of the "#" at the start
  # of the header line.
  with open(filename, 'r') as f:
    data = f.readlines()

  # Save the whole thing for possible debugging.
  ref.data = data

  # Save the meta data in the data structure in case the user wants it.
  ref.meta = data[0]

  # Get rid of the "#" at the start of the header line.
  if data[1][0:6] == '#Label':
    data[1] = data[1][1:]

  # Convert the list to a string
  ds = ''.join( data )

  # Read the string into a Pandas dataframe and store in the data structure.
  ref.df = pd.read_csv( StringIO(ds), comment='#')

  # Read the last line of the file into a Pandas dataframe and store in the data structure.
  # This line has the total error for several columns.
  ref.total_error = pd.read_csv( StringIO( data[-1] ), names=ref.df.columns ).dropna(axis=1)
  ref.total_error.replace( to_replace="#Total error", value=ref.root_filename, inplace=True )

  # Return the data structure.
  return ref

In [ ]:
# Gather test reference files.
if __name__ == '__main__':
  files = glob(f'{test_data_dir}/*-ref.txt')
  display(files)

In [ ]:
# Test with a single ref file.
if __name__ == "__main__":
  ref = read_metashape_reference_file_into_dataframe( files[0] )
  display(ref.df.head(3), ref.df.tail(3), ref.total_error)

### def read_metashape_reference_file_total_errors()

In [ ]:
def read_metashape_reference_file_total_errors( filename ):
  df = read_metashape_reference_file_into_dataframe( filename )
  s = df.df['Label'][0].split('.')[0]
  dtime = dt.datetime.strptime(s, '%Y%m%d-%H%M%S')
  df.total_error['datetime'] = dtime
  return df.total_error

In [ ]:
# Test read_metashape_reference_file_total_errors()
if __name__ == '__main__':
  df = read_metashape_reference_file_total_errors( ref_test_files[0] )
  display(df)

In [ ]:
# Test read_metashape_reference_file_total_errors() with multiple files
if __name__ == '__main__':
  df = pd.DataFrame() # read_metashape_reference_file_total_errors( ref_test_files[0] )
  for f in ref_test_files:
    dfn = read_metashape_reference_file_total_errors( f )
    df = pd.concat( [df, dfn] )
  display(df)

In [ ]:
class MetaShape_ref_total_errors():
  def __init__(self):
    self.path   = None
    self.df     = None

### def read_metashape_reference_dir_total_errors()

In [ ]:
def read_metashape_reference_dir_total_errors(
    dir_path,         # Path to MetaShape reference data files.
    mask='*-ref.txt'  # Mask to select the reference files.
    ) -> object:      # Class with path and dataframe attributes
  df = pd.DataFrame()
  rv = MetaShape_ref_total_errors()
  rv.path = dir_path
  for f in glob(f'{dir_path}/{mask}'):
    dfn = read_metashape_reference_file_total_errors( f )
    df = pd.concat( [df, dfn] )
  v = df.pop('datetime')            # Move 'datetime' to be the second column
  df.insert(1, 'datetime', v)
  df.reset_index(drop=True, inplace=True)
  rv.df = df
  return rv

#### Test def read_metashape_reference_dir_total_errors()

In [ ]:
# Test read_metashape_reference_file_total_errors() with multiple files
if __name__ == '__main__':
  rv = read_metashape_reference_dir_total_errors( test_data_dir )
  display(rv.path, rv.df)

### class MetaShape_Cal_Data()

In [ ]:
class MetaShape_Cal_Data():
  def __init__(self):
    self.filename   = None
    self.datetime   = None
    self.projection = None
    self.units      = None
    self.pixel_size = None
    self.f          = None
    self.cx         = None
    self.cy         = None
    self.width      = None
    self.height     = None
    self.error      = None

### def read_metashape_camera_cal_file()

In [ ]:
def read_metashape_camera_cal_file(
    filename,              # The MetaShape xml calibration file to read.
    pixel_size  = 5.5e-6,  # Pixel size in meters.
    units = 'pixels',      # Can be "pixels", "m", or "mm"
    rv = 'dict',           # R value. Can be "dict" or "df" or "class"
    debug=False            #
    ) -> dict:             #
  """
  Read a Metashape xml camera calibration file into a dictionary. Values
  can be converted from pixels to meters, or millimeters. Be sure to
  specify the pixel size in meters.
  """
  cd = MetaShape_Cal_Data()    # class to hold data.
  cal = { "filename" : Path(filename),   # filename.split("/")[-1],
         "pixel_size": pixel_size,
          "units"     : units
        }
  cd.filename   = Path(filename)
  cd.pixel_size = pixel_size
  cd.units      = units
  try:
    tree = ET.parse( filename )
    root = tree.getroot()
  except:
    cd.error = f"Et.parse( {filename} ) failed."
    return cd

  if debug:
    print(f'debug: root.tag:{root.tag}')
  if root.tag == 'calibration':
    for child in root:
      if debug:
        print(f'debug: {child.tag:10s} = {child.text}')
      if child.tag == 'projection':
        cd.projection = cal[child.tag] = child.text
      elif child.tag == 'date':
        cd.datetime = cal[child.tag] = dt.datetime.strptime( child.text,"%Y-%m-%dT%H:%M:%SZ" )
      else:
        cal[child.tag] = float(child.text)

    if units != 'pixels':
      if units   == 'm':
        scale = 1.0
      elif units == 'mm':
        scale = 1000.0

      cd.f      = cal['f' ]      * cd.pixel_size * scale
      cd.cx     = cal['cx']     * cd.pixel_size * scale
      cd.cy     = cal['cy']     * cd.pixel_size * scale
      cd.width  = cal['width']  * cd.pixel_size * scale
      cd.height = cal['height'] * cd.pixel_size * scale

      for k in ['f', 'cx', 'cy', 'width', 'height']:
        cal[k] = cal[k] * cal['pixel_size'] * scale

    if rv == 'class':
      cd.error = 'None'
      return cd
    elif rv == 'dict':
      return cal
    elif rv == 'df':
      df = pd.DataFrame( [ cal ] )
      return df
  else:
    cd.error = f"Not a MetaShape calibration file, lacks <calibration> tag."
    return cd

#### Test def read_metashape_camera_cal_file() with rv='class'

In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='class' )

In [ ]:
if __name__ == "__main__":
  print(
      f"{cal.error      = }\n"
      f"{cal.datetime   = }\n"
      f"{cal.filename   = }\n"
      f"{cal.pixel_size = }\n"
      f"{cal.width      = }\n"
      f"{cal.height     = }\n"
      f"{cal.projection = }\n"
      f"{cal.units      = }\n"
      f"{cal.f          = }\n"
      f"{cal.cx         = }\n"
      f"{cal.cy         = }\n"
      )

  # Dump contents via keys and values.
  for k in cal.__dict__.keys():
    print(f'{k:15s}= {cal.__dict__[k]}')

  # Dump all the parts of the filename
  display(cal.filename.parts)

  # Show the filename without the path
  print(f"Filename without the path: {cal.filename.name}")
  print(f"Pathname without the file: {cal.filename.parent}")

In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df' )
  display(cal)

In [ ]:
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, debug=True )
  display(cal)

In [ ]:
# Test with the wrong file, a non xml file.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( inpho_filename, debug=True )
  display(cal.error)

In [ ]:
# Test with an xml file which is not a Metashape cal file.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( opencv_filename, units='mm' )
  display(cal.error)

#### Test read_metashape_camera_cal_file() with rv='df

In [ ]:
# Test with df returns.
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df' )
  display(cal)

### def read_openCV_camera_cal_file( filename, debug=False )

In [ ]:
class read_openCV_camera_cal_file:
  """
  Read an OpenCV Camera xml Calibration file.
  """
  def __init__(self, filename ):
    tree = ET.parse( filename )
    root = tree.getroot()
    tree.getroot()

    #self.filename = filename.split("/")[-1]
    filepath = Path( filename )
    self.filename  = filepath  #filepath.name
    self.datetime = root.findall('calibration_Time')[0].text
    self.image_Width = root.findall('image_Width')[0].text
    self.image_Height = root.findall('image_Height')[0].text

    self.Camera_Matrix_rows = int(root.findall('Camera_Matrix/rows')[0].text)
    self.Camera_Matrix_cols = int(root.findall('Camera_Matrix/cols')[0].text)
    self.Camera_Matrix_data = root.findall('Camera_Matrix/data')[0].text
    lst = root.findall('Camera_Matrix/data')[0].text.strip()
    x = re.split('[ \n]+', lst )
    self.Camera_Matrix_data = np.array(x, dtype=float)
    self.Camera_Matrix_data.reshape( self.Camera_Matrix_rows,self.Camera_Matrix_cols)

    self.Distortion_Coefficients_rows = int(root.findall('Distortion_Coefficients/rows')[0].text)
    self.Distortion_Coefficients_cols = int(root.findall('Distortion_Coefficients/cols')[0].text)
    lst = root.findall('Distortion_Coefficients/data')[0].text.strip()
    x = re.split('[ \n]+', lst )
    self.Distortion_Coefficients_data = np.array(x, dtype=float)

    # Add as a MetaShape class.
    self.metashape = MetaShape_Cal_Data()
    self.metashape.units        = 'pixels'
    self.metashape.pixel_size   = 5.5e-6
    self.metashape.projection   = 'frame'
    self.metashape.filename     = filepath
    self.metashape.datetime     = self.datetime
    self.metashape.width        = self.image_Width
    self.metashape.height       = self.image_Height
    self.metashape.f            = self.Camera_Matrix_data[0]
    self.metashape.cx           = self.Camera_Matrix_data[2]
    self.metashape.cy           = self.Camera_Matrix_data[5]
    self.metashape.k1           = self.Distortion_Coefficients_data[0]
    self.metashape.k2           = self.Distortion_Coefficients_data[1]
    self.metashape.k3           = self.Distortion_Coefficients_data[4]
    self.metashape.p1           = self.Distortion_Coefficients_data[3]
    self.metashape.p2           = self.Distortion_Coefficients_data[2]

#### Test read_openCV_camera_cal_file()



In [ ]:
if __name__ == "__main__":
  cal = read_openCV_camera_cal_file(opencv_filename)

  # Dump the data.
  print("\n**** openCV elements. ****")
  for k, v in cal.__dict__.items():
    print(f'{k:32s} = {v}')

  # Dump the metashape data.
  print("\n**** MetaShape elements. ****")
  for k, v in cal.metashape.__dict__.items():
    print(f'{k:32s} = {v}')

  print("\n**** Example reading elements directly ****")
  print(f'{cal.metashape.width = }')

### def metashape_cal_to_tsai()

In [ ]:
def metashape_cal_to_tsai( df,               # MetaShape cal dataframe.
                          save=True,         # True to save tasi to a file.
                           tsai_file='',     # An optional filename to write the tsai data too.
                           path=".",         # Path to save the file to.
                           return_str=False, # True if you want the tsai string returned.
                           debug=False       # True debug tis module.
                           ) -> str:         # Returns the entire tsai cal string.

  # Compute the tsai values from the Metashape cal dataframe.
  fu  = fv = df.f[0]
  cu  = df.width[0]  / 2 + 0.5 - 1.0 + df.cx[0]
  cv  = df.height[0] / 2 + 0.5 - 1.0 + df.cy[0]
  k1  = df.k1[0];   k2 = df.k2[0];   p1 = df.p1[0];   p2 = df.p2[0]

  # Generate tsai string.
  tsai_str = \
       "VERSION_4\nPINHOLE\n"\
      f"{fu = :15.12f}\n{fv = :15.12f}\n{cu = :15.12f}\n{cv = :15.12f}\n"\
      "u_direction = 1  0  0\n"\
      "v_direction = 0  1  0\n"\
      "w_direction = 0  0  1\n"\
      "C = 0 0 0\nR = 1 0 0 0 1 0 0 0 1\n"\
      "pitch = 1.0\nTSAI\n"\
      f"{k1 = :15.12f}\n{k2 = :15.12f}\n{p1 = :15.12f}\n{p2 = :15.12f}\n"

  # Save it to a file.  Generate the file name from the agisoft filename.
  if save:
    if tsai_file:
      tsai_fn = Path(path+"/"+tsai_file)
      if debug:
        print(f'Debug: {tsai_fn=}')
    else:
      # tsai_fn = df.filename[0].split(".")[0]+".tsai"
      tsai_fn = Path(path+"/"+df.filename[0].name).with_suffix('.tsai')
      if debug:
        print(f'Debug: {tsai_fn=}')
      # tsai_fn.with_suffix('.tsai')
    with open( tsai_fn, 'w') as tsai_file:
      tsai_file.write( tsai_str )

  if debug:
    print(f'debug: {tsai_str:s}')

  # Return the tsai string if return_str=True
  if return_str:
    return tsai_str

#### Test def metashape_cal_to_tsai()

In [ ]:
# test metashape_cal_to_tsai( cal )
if __name__ == "__main__":
  cal = read_metashape_camera_cal_file( metashape_cal_filename, units='m', rv='df', debug=True )
  display(cal)
  print(cal.filename)
  metashape_cal_to_tsai( cal )

In [ ]:
# Test with user set file name.
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, tsai_file="junk.tsai" )

In [ ]:
# Test with automatically generated filename
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, debug=True )

In [ ]:
# Test with user set path. Write the test file to /tmp
if __name__ == "__main__":
  metashape_cal_to_tsai( cal, path="/tmp" )

## Experimentation Code

In [ ]:
if __name__ == "__main__" and False:
  fn = Path("/dddd"+'/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/we-1500-A1-opencv-camcal.xml')
  display(fn.parts)

In [ ]:
if __name__ == "__main__" and False:
  fn = fn.with_suffix('.tsai')
  display(fn)

In [ ]:
# Testing cell.
if __name__ == "__main__" and False:
  filename = Path('/content/drive/MyDrive/Colab Notebooks/WWpylib/test_data/we-1500-A1-agisoft-camcal.xml')
  wp = PureWindowsPath(r"C:\Users\mstuding\OneDrive - NASA\GitHub_repositories\ATM-SfM-Production")
  display(wp.parts)

## References

1. [Medium Article: pathlib](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) path(), PurePosixPath() and PureWindowsPath()